## ISYE 6501 Homework 11

#### Question 15.2

In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization
problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional
requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem
with real data. The data is given in the file diet.xls.

1. Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the
    maximum and minimum daily nutrition constraints, and solve it using PuLP. Turn in your code
    and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs,
    oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)
    
2. Please add to your model the following constraints (which might require adding more variables)
    and solve the new model:
    
        a. If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will
        need two variables for each food i: whether it is chosen, and how much is part of the diet.
        You’ll also need to write a constraint to link them.)
        
        b. Many people dislike celery and frozen broccoli. So at most one, but not both, can be
        selected.
        
        c. To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be
        selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered
        meat?), just call it whatever you think is appropriate – I want you to learn how to write this
        type of constraint, but I don’t really care whether we agree on how to classify foods!]


In [1]:
from pulp import *
import numpy as np
import pandas as pd

### Preparing the data

In [2]:
#loading data

data=pd.read_excel("diet.xls")

In [3]:
data.shape

(67, 14)

In [4]:
data.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [5]:
data.tail()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [6]:
data=data.loc[:63,:]

In [7]:
data.shape

(64, 14)

In [8]:
data.tail()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0


In [9]:
data.isna().sum()

Foods              0
Price/ Serving     0
Serving Size       0
Calories           0
Cholesterol mg     0
Total_Fat g        0
Sodium mg          0
Carbohydrates g    0
Dietary_Fiber g    0
Protein g          0
Vit_A IU           0
Vit_C IU           0
Calcium mg         0
Iron mg            0
dtype: int64

### Creating an LP problem

In [52]:
prob = LpProblem("Diet Problem",LpMinimize)

### Creating dictionary of nutritient for each food

In [53]:
foods=list(data["Foods"])

In [54]:
costs=dict(zip(foods,data['Price/ Serving']))

calories=dict(zip(foods,data['Calories']))
cholesterol=dict(zip(foods,data['Cholesterol mg']))

fat=dict(zip(foods,data['Total_Fat g']))
sodium=dict(zip(foods,data['Sodium mg']))

carbohydrates=dict(zip(foods,data['Carbohydrates g']))
dietary_fiber=dict(zip(foods,data['Dietary_Fiber g']))

protein=dict(zip(foods,data['Protein g']))
vitamin_A=dict(zip(foods,data['Vit_A IU']))

vitamin_C=dict(zip(foods,data['Vit_C IU']))
calcium=dict(zip(foods,data['Calcium mg']))

iron=dict(zip(foods,data['Iron mg']))

### Declaration of lower bound, for non-negative quantities of food choice 

In [55]:
amount_vars = LpVariable.dicts("Amounts",foods,lowBound=0,cat='Continuous')


### Objective Function

In [56]:
prob += lpSum([costs[i]*amount_vars[i] for i in foods])

### Nutrition Constraints

In [57]:
#calories
prob += lpSum([calories[f] * amount_vars[f] for f in foods]) >= 1500
prob += lpSum([calories[f] * amount_vars[f] for f in foods]) <= 2500

#cholesterol
prob += lpSum([cholesterol[f] * amount_vars[f] for f in foods]) >= 30
prob += lpSum([cholesterol[f] * amount_vars[f] for f in foods]) <= 240

#fat
prob += lpSum([fat[f] * amount_vars[f] for f in foods]) >= 20
prob += lpSum([fat[f] * amount_vars[f] for f in foods]) <= 70

#sodium
prob += lpSum([sodium[f] * amount_vars[f] for f in foods]) >= 800
prob += lpSum([sodium[f] * amount_vars[f] for f in foods]) <= 2000

#carbs
prob += lpSum([carbohydrates[f] * amount_vars[f] for f in foods]) >= 130
prob += lpSum([carbohydrates[f] * amount_vars[f] for f in foods]) <= 450

#fiber
prob += lpSum([dietary_fiber[f] * amount_vars[f] for f in foods]) >= 125
prob += lpSum([dietary_fiber[f] * amount_vars[f] for f in foods]) <= 250

#protein
prob += lpSum([protein[f] * amount_vars[f] for f in foods]) >= 60
prob += lpSum([protein[f] * amount_vars[f] for f in foods]) <= 100

#vitamin A
prob += lpSum([vitamin_A[f] * amount_vars[f] for f in foods]) >= 1000
prob += lpSum([vitamin_A[f] * amount_vars[f] for f in foods]) <= 10000

#vitamin C
prob += lpSum([vitamin_C[f] * amount_vars[f] for f in foods]) >= 400
prob += lpSum([vitamin_C[f] * amount_vars[f] for f in foods]) <= 5000

#calcium
prob += lpSum([calcium[f] * amount_vars[f] for f in foods]) >= 700
prob += lpSum([calcium[f] * amount_vars[f] for f in foods]) <= 1500

#iron
prob += lpSum([iron[f] * amount_vars[f] for f in foods]) >= 10
prob += lpSum([iron[f] * amount_vars[f] for f in foods]) <= 40

### Optimization for Part 1

In [58]:
prob.solve()

print("---------The solution for Part 1 ----------")
print(40*"-")
for var in prob.variables():
    if var.varValue > 0:
        print(str(var.varValue)+" units of "+str(var).replace('Amounts_','') )
        
        
print(40*"-")
print("Total cost = ",value(prob.objective))

---------The solution for Part 1 ----------
----------------------------------------
52.64371 units of Celery,_Raw
0.25960653 units of Frozen_Broccoli
63.988506 units of Lettuce,Iceberg,Raw
2.2929389 units of Oranges
0.14184397 units of Poached_Eggs
13.869322 units of Popcorn,Air_Popped
----------------------------------------
Total cost =  4.337116797399999


### Optimization for Part 2

In [59]:
#a binary variable for whether a food is chosen
binary_var = LpVariable.dicts("select_food",foods,0,1,LpBinary)

In [60]:
#constraint a
for food in foods:
    #if a food is selected, then a minimum of 1/10 serving must be chosen
    prob += amount_vars[food] >= 0.1 * binary_var[food]
    
    #if a food is not selected, the binary variable must be 0, and food must be 0
    prob += amount_vars[food] <= 1000000 * binary_var[food]
    
#constraint b
#at most one out of celery and frozen broccoli can be selected, but not both.
prob += binary_var['Frozen Broccoli'] + binary_var['Celery, Raw'] <= 1 

#constraint c
#at least 3 kinds of meat/poultry/fish/eggs must be selected

prob +=   binary_var['Roasted Chicken'] + binary_var['Poached Eggs'] \
        + binary_var['Scrambled Eggs'] + binary_var['Bologna,Turkey'] \
        + binary_var['Frankfurter, Beef'] + binary_var['Ham,Sliced,Extralean'] \
        + binary_var['Kielbasa,Prk'] + binary_var['Pizza W/Pepperoni'] \
        + binary_var['Hamburger W/Toppings'] \
        + binary_var['Hotdog, Plain'] + binary_var['Pork'] \
        + binary_var['Sardines in Oil'] + binary_var['White Tuna in Water'] \
        + binary_var['Chicknoodl Soup'] + binary_var['Splt Pea&Hamsoup'] \
        + binary_var['Vegetbeef Soup'] + binary_var['Neweng Clamchwd'] \
        + binary_var['New E Clamchwd,W/Mlk'] + binary_var['Beanbacn Soup,W/Watr'] >= 3


In [61]:
prob.solve()

print("---------The solution for Part 2 ----------")
print(45*"-")
for var in prob.variables():
    if var.varValue > 0 and "select_food" not in var.name:
        print(str(var.varValue)+" units of "+str(var).replace('Amounts_','') )
        
print(45*"-")
print("Total cost = ",value(prob.objective))

---------The solution for Part 2 ----------
---------------------------------------------
42.399358 units of Celery,_Raw
0.1 units of Kielbasa,Prk
82.802586 units of Lettuce,Iceberg,Raw
3.0771841 units of Oranges
1.9429716 units of Peanut_Butter
0.1 units of Poached_Eggs
13.223294 units of Popcorn,Air_Popped
0.1 units of Scrambled_Eggs
---------------------------------------------
Total cost =  4.512543427000001
